In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

In [61]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

!ollama pull llama3.2

OLLAMA_MODAL_URL = "http://localhost:11434/v1"
MODEL_NAME = "llama3.2"
API_KEY = "ollama"
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


def get_messages(url):
    web_page = Website(url)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(web_page)}   
    ]

    return messages

def display_summarize_text(url):
    messages = get_messages(url)
    #openai_use_to_local_llama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

    #response = openai_use_to_local_llama.chat.completions.create(model="llama3.2", messages=messages)

    #print(messages)
    open_ai_local_llama = OpenAI(base_url=OLLAMA_MODAL_URL, api_key=API_KEY)
    response = open_ai_local_llama.chat.completions.create(model=MODEL_NAME, messages=messages)
    display(Markdown(response.choices[0].message.content))
     



pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [62]:
display_summarize_text("https://www.forbes.com/profile/elon-musk/")

**Elon Musk: A Business Magnate & Tech Industry Icon**
### Net Worth:
* $409B as of 6/19/25 (Reflects change since 5 pm ET of prior trading day)

### Companies and Investments:

* Tesla (holds stake in)
* SpaceX (owner, with a valuation of $350 billion based on December 2024 private share sale)

### News & Developments:
* **elonmusk**, **Forbes Lists**
	+ The Richest Person In Every State (2025) - #1
	+ Forbes 400 (2024) - #1
	+ Innovative Leaders (2019) - #25
	+ Powerless People (2018) - #12
	+ Richest in Tech (2017)
	+ Global Game Changers (2016)

### Personal Stats:

* Age: 53
* Source of Wealth: Tesla, SpaceX, Self-Made
* Self- Made Score: 8
* Philanthropy Score: 1
* Residence: Austin, Texas

### Photos:
Photo by Martin Schoeller for Forbes

In [60]:
from openai import OpenAI
!ollama pull llama3.2
# Step 1: Create your prompts

system_prompt = "You are an assistant whose task is to read an email (official puprpose) and suggest a subject line for the email "
user_prompt = """
Provide a sample line for below mail
To:
Embassy of Germany
Consular Section
17, Nyaya Marg, Chanakyapuri
New Delhi - 110021

Date: June 16, 2025

Dear Sirs,

This letter is to present Ms. Ananya Rao, Business Development Manager at InnovaSoft Solutions Pvt. Ltd.

The purpose of Ms. Ananya Rao’s visit to Germany is to represent InnovaSoft Solutions Pvt. Ltd. on a business development and client onboarding trip with our German partner firm.

The relevant information pertaining to this visit is as follows:

Name of applicant: Ananya Rao

Nationality: Indian

Passport #: M1234567

Date of issue of passport: January 10, 2022

Expiry date: January 09, 2032

Place of birth: Bengaluru, India

Date of birth: April 15, 1991

Occupation: Business Development Manager

Name of inviting company: TechHub GmbH, Berlin

Period of stay: July 5, 2025 – July 19, 2025

We kindly ask you to grant Ms. Ananya Rao the appropriate short-term Schengen business visa for the above-mentioned purpose. Please be advised that InnovaSoft Solutions Pvt. Ltd. will accept full financial responsibility for Ms. Ananya Rao during this visit.

Thank you.

Yours truly,
Rakesh Mehta
Director – International Affairs
InnovaSoft Solutions Pvt. Ltd.

"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}] # fill this in

# Step 3: Call llama 3.2 running on local system

openai_use_to_local_llama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = openai_use_to_local_llama.chat.completions.create(model="llama3.2", messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 
Here are a few suggested subject lines for the email:

1. Request for Business Visa Application - Ms. Ananya Rao
2. Appointment of Representative: Ms. Ananya Rao (India) - Schengen Visa
3. Invitation to Germany: InnovaSoft Solutions Pvt. Ltd. Employee Visit
4. Visa Application R